In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from networks.inception import InceptionResnet, SupIncepResnet
from networks.simple_cnn import BaselineCNNClassifier
from networks.resnet_big import SupCEResNet, SupConResNet, LinearClassifier
import torch
from torchsummary import summary
from thop import profile
import numpy as np
import time

In [3]:
def measure_time_gpu(model, device, rep):
    model = model.to(device=device)
    dummy_input = torch.randn(1, 1, 29, 29, dtype=torch.float).to(device)
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = rep
    timings=np.zeros((repetitions,1))
    #GPU-WARM-UP
    for _ in range(100):
        _ = model(dummy_input)
    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):
            starter.record()
            _ = model(dummy_input)
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time
    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    return mean_syn, std_syn


In [4]:
def measure_time_cpu(model, device, rep = 10):
    model = model.to(device=device)
    x = torch.rand((1, 1, 29, 29), device=device)
    timings=np.zeros((rep,1))
    for i in range(rep):    
        start_time = time.time()
        out = model(x)
        timings[i] = time.time() - start_time
    mean_syn = np.sum(timings) / rep
    std_syn = np.std(timings)
    return mean_syn, std_syn

In [ ]:
incep = SupIncepResnet(num_classes=5)
measure_time_gpu(incep, 'cuda', rep=1000)

In [ ]:
baseline_model = SupCEResNet(name='resnet18', num_classes=5)
measure_time_gpu(baseline_model, 'cuda', rep=1000)

In [ ]:
x = torch.rand((1, 1, 29, 29), device='cpu')
baseline_model = baseline_model.to(device='cpu')
macs, params = profile(baseline_model, inputs=(x, ))
print('MACs (G): ', macs/1000**3)
print('Params (M): ', params/1000**2)

In [ ]:
x = torch.rand((1, 1, 29, 29), device='cpu')
incep = incep.to(device='cpu')
macs, params = profile(incep, inputs=(x, ))
print('MACs (G): ', macs/1000**3)
print('Params (M): ', params/1000**2)

In [ ]:
summary(baseline_model.cuda(), (1, 29, 29))

In [ ]:
summary(incep.cuda(), (1, 29, 29))